In [ ]:
from langchain.document_loaders.rspace import RSpaceLoader
import os

from dotenv import load_dotenv

load_dotenv()

GHA_TOKEN = os.getenv('GHA')


In [ ]:
rspace_rfp="SD1932247"
walks= "NB182183"

In [ ]:
loader = RSpaceLoader(global_id=walks)
docs=loader.load()
docs[0].to_json()
len(docs)


In [ ]:
len(docs)
summaries = [d.page_content[0:100] for d in  docs]
print('\n----\n'.join(summaries))


In [ ]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores.utils import filter_complex_metadata
filtered = filter_complex_metadata(docs)

In [ ]:
c = CharacterTextSplitter(chunk_size=100, chunk_overlap=20)
r = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=40)
all_chunks = r.split_documents(filtered)
all_chunks[0]
len(all_chunks)

    

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores.utils import filter_complex_metadata


In [ ]:
filtered = filter_complex_metadata(all_chunks)
filtered[0].metadata
len(filtered)
filtered[2]

In [ ]:
v_store = Chroma.from_documents(documents=filtered, embedding=OpenAIEmbeddings())
v_store.

In [ ]:
results = v_store.similarity_search(query = "alignment", k=4, score_threshold=0.8)
results

In [ ]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo


In [ ]:
results = sqretriever.get_relevant_documents("created by richarda23", verbose=True)
results


In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models  import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.9, verbose=True)
retriever = v_store.as_retriever( search_kwargs={'k':4})
vector_results = retriever.get_relevant_documents(query="what is my longest walk")

v_store.persist()
vector_results


In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm,chain_type='stuff',retriever=retriever,verbose=True )

result = qa_chain({"query": "what walks has the weather been good on"})
result

In [ ]:
result = qa_chain({"query": "what tools does RSpace integrate with  "})
result

In [ ]:
result

In [ ]:
#v_store.delete_collection()